In [424]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout

In [425]:
merged_data_path = '../data/v2/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

In [426]:
# K-MEANS

# Prepare the list of unique genres
unique_genres = list(set([genre for genres in data['genres'] for genre in genres]) | set([genre for genres in data['favourite_genres'] for genre in genres]))

# Convert the genres to a matrix of TF-IDF features
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(unique_genres)

# Apply K-means clustering
num_clusters = 100  # Adjust this value according to your needs
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init='auto')
kmeans.fit(vectorized)
labels = kmeans.labels_

# Group the genres based on the cluster labels
clustered_genres = {}
for i, label in enumerate(labels):
    if label not in clustered_genres:
        clustered_genres[label] = []
    clustered_genres[label].append(unique_genres[i])

# Map genres to their cluster label
genre_to_cluster = {genre: label for label, genres in clustered_genres.items() for genre in genres}

# Define the function for mapping genres to simpler forms
def map_genre(genre):
    cluster_label = genre_to_cluster[genre]
    representative_genre = clustered_genres[cluster_label][0]  # Use the first genre in the cluster as the representative
    return representative_genre

# Save the genre_to_cluster and clustered_genres dictionaries to use for new data prediction
with open('../models/model2_genre_to_cluster.pickle', 'wb') as f:
    pickle.dump(genre_to_cluster, f)

with open('../models/model2_clustered_genres.pickle', 'wb') as f:
    pickle.dump(clustered_genres, f)

# Apply the mapping function to both 'genres' and 'favourite_genres' columns
data['genres'] = data['genres'].apply(lambda x: [map_genre(genre) for genre in x])
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: [map_genre(genre) for genre in x])

# Delete not unique genres
data['genres'] = data['genres'].apply(lambda x: list(set(x)))
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: list(set(x)))


In [427]:
# count unique genres
unique_genres = set()
for genres in data['genres']:
    unique_genres.update(genres)
for genres in data['favourite_genres']:
    unique_genres.update(genres)
print(len(unique_genres))

# how many tracks are per genre
genre_count = {}
for genres in data['genres']:
    for genre in genres:
        if genre not in genre_count:
            genre_count[genre] = 0
        genre_count[genre] += 1

genre_count = sorted(genre_count.items(), key=lambda x: x[1], reverse=True)
genre_count

100


[('poezja spiewana', 5548),
 ('rock quebecois', 4689),
 ('classic luk thung', 3074),
 ('greek pop', 2523),
 ('pakistani folk', 1630),
 ('hard trance', 1141),
 ('dark wave', 988),
 ('country blues', 987),
 ('italo dance', 686),
 ('rap francais', 658),
 ('polish post-punk', 589),
 ('comic metal', 577),
 ('modern alternative rock', 557),
 ('estonian hip hop', 551),
 ('israeli singer-songwriter', 544),
 ('jazz pop', 532),
 ('british dance band', 479),
 ('classic israeli pop', 471),
 ('trap', 381),
 ('latin classical', 374),
 ('new orleans funk', 319),
 ('classic canadian rock', 295),
 ('glam punk', 295),
 ('okc indie', 250),
 ('neo classical metal', 221),
 ('progressive house', 215),
 ('disco polo', 180),
 ('barrelhouse piano', 176),
 ('vocal house', 176),
 ('modern power pop', 139),
 ('reggaeton cristiano', 125),
 ('old school rap francais', 120),
 ('edm', 104),
 ('world fusion', 102),
 ('australian pop', 102),
 ('big room', 99),
 ('swedish electronic', 89),
 ('forro tradicional', 87),
 (

In [428]:
# balance set
skipped_data = data[data["skipped"] == 1]
not_skipped_data = data[data["skipped"] == 0]

# Split the dataset into subsets based on class labels
print("skipped_data", skipped_data.shape)
print("not_skipped_data", not_skipped_data.shape)

# Calculate the minimum number of samples among all classes
min_samples = min(len(skipped_data), len(not_skipped_data))

# Randomly select samples from each class subset to match the minimum number of samples
skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# Merge the balanced subsets to create the final balanced dataset
balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

skipped_data (3621, 3)
not_skipped_data (6260, 3)


In [429]:
# Combine genres and favourite_genres
mlb = MultiLabelBinarizer()

data['combined_genres'] = data.apply(lambda x: x['genres'] + x['favourite_genres'], axis=1)
mlb.fit(data['combined_genres'])

encoded_favourite_genres = mlb.transform(balanced_data['favourite_genres'])
encoded_genres = mlb.transform(balanced_data['genres'])

# save mlb to file
with open('../models/model2_mlb.pickle', 'wb') as f:
    pickle.dump(mlb, f)


In [430]:
X = np.concatenate([encoded_genres, encoded_favourite_genres], axis=1)

# Extract the labels
y = balanced_data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
# print("Y_train not skipped", np.count_nonzero(y_train == 0))
# print("Y_train skipped", np.count_nonzero(y_train == 1))
# print("Y_train skipped %", np.count_nonzero(y_train == 1) / len(y_train))

X (7242, 200)


In [431]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(10 * 2, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(
      optimizer=Adam(),
      loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [432]:
epochs = 50
batch_size = 32

model = build_model(None)

model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

# save model to file
model.save('../models/model2.h5')

Epoch 1/50
182/182 - 2s - loss: 0.6944 - accuracy: 0.5032 - val_loss: 0.6904 - val_accuracy: 0.5452 - 2s/epoch - 13ms/step
Epoch 2/50
182/182 - 1s - loss: 0.6928 - accuracy: 0.5068 - val_loss: 0.6878 - val_accuracy: 0.5721 - 1s/epoch - 7ms/step
Epoch 3/50
182/182 - 1s - loss: 0.6878 - accuracy: 0.5251 - val_loss: 0.6815 - val_accuracy: 0.5880 - 1s/epoch - 7ms/step
Epoch 4/50
182/182 - 1s - loss: 0.6820 - accuracy: 0.5584 - val_loss: 0.6726 - val_accuracy: 0.6149 - 1s/epoch - 8ms/step
Epoch 5/50
182/182 - 1s - loss: 0.6761 - accuracy: 0.5683 - val_loss: 0.6652 - val_accuracy: 0.6094 - 1s/epoch - 7ms/step
Epoch 6/50
182/182 - 1s - loss: 0.6696 - accuracy: 0.5835 - val_loss: 0.6587 - val_accuracy: 0.6156 - 1s/epoch - 7ms/step
Epoch 7/50
182/182 - 1s - loss: 0.6654 - accuracy: 0.5938 - val_loss: 0.6541 - val_accuracy: 0.6266 - 1s/epoch - 6ms/step
Epoch 8/50
182/182 - 1s - loss: 0.6600 - accuracy: 0.6066 - val_loss: 0.6504 - val_accuracy: 0.6342 - 1s/epoch - 6ms/step
Epoch 9/50
182/182 - 1s

In [433]:
print("TEST")
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
print("Classification report:\n", classification_report(y_test, y_pred_classes))

print("TRAIN")
y_pred = model.predict(X_train)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_train, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
print("Classification report:\n", classification_report(y_train, y_pred_classes))


TEST
46/46 [==============================] - 0s 3ms/step
Accuracy: 0.6549344375431332
Confusion matrix:
 [[471 258]
 [242 478]]
Classification report:
               precision    recall  f1-score   support

           0       0.66      0.65      0.65       729
           1       0.65      0.66      0.66       720

    accuracy                           0.65      1449
   macro avg       0.66      0.65      0.65      1449
weighted avg       0.66      0.65      0.65      1449

TRAIN
182/182 [==============================] - 0s 2ms/step
Accuracy: 0.707578111513896
Confusion matrix:
 [[2005  887]
 [ 807 2094]]
Classification report:
               precision    recall  f1-score   support

           0       0.71      0.69      0.70      2892
           1       0.70      0.72      0.71      2901

    accuracy                           0.71      5793
   macro avg       0.71      0.71      0.71      5793
weighted avg       0.71      0.71      0.71      5793

